# ASTR 119 Final Project #3 - Group F - f105w

This project was created as a collaboration of all group members. No specific member takes sole responsibility for any part of the project. We all worked on it collectively.

Importing packages

In [ ]:
import numpy as np
import sep

In [ ]:
from astropy.io import fits
import matplotlib.pyplot as plt
from matplotlib import rcParams
%matplotlib inline

rcParams['figure.figsize'] = [10.,8.]

In [ ]:
Reading the image into a standard 2-D numpy array

In [ ]:
hdu_list = fits.open("C:\\Users\\peter\\Downloads\\hlsp_hudf12_hst_wfc3ir_udfmain_f105w_v1.0_drz.fits")
data = hdu_list[0].data

Display the original image

In [ ]:
m, s = np.mean(data), np.std(data)
plt.imshow(data, interpolation='nearest', cmap='gray', vmin=m-s, vmax=m+s, origin='lower')
plt.colorbar();

Measure a spatially varying background on the image

In [ ]:
data = data.byteswap().newbyteorder()
bkg = sep.Background(data)

Get a "global" mean and noise of the image background

In [ ]:
print(bkg.globalback)
print(bkg.globalrms)

Evaluate the background as a 2-D array as same the size as original image

In [ ]:
bkg_image = bkg.back()

Display the background

In [ ]:
plt.imshow(bkg_image, interpolation='nearest', cmap='gray', origin='lower')
plt.colorbar();

Evaluate the background noise as 2-d array as the same size as original image

In [ ]:
bkg_rms = bkg.rms()

Display the background noise as an image

In [ ]:
plt.imshow(bkg_rms, interpolation='nearest', cmap='gray', origin='lower')
plt.colorbar();

Subtract the background from the original image

In [ ]:
data_sub = data - bkg

Extract the objects from the data_sub image

In [ ]:
objects = sep.extract(data_sub, 1.5, err=bkg.globalrms)

Return how many objects were detected

In [ ]:
len(objects)

Circle the objects found on the background-subtracted image with a red ellipse and display the final image

In [ ]:
from matplotlib.patches import Ellipse

# plot background-subtracted image
fig, ax = plt.subplots()
m, s = np.mean(data_sub), np.std(data_sub)
im = ax.imshow(data_sub, interpolation='nearest', cmap='gray',
               vmin=m-s, vmax=m+s, origin='lower')

# plot an ellipse for each object
for i in range(len(objects)):
    e = Ellipse(xy=(objects['x'][i], objects['y'][i]),
                width=6*objects['a'][i],
                height=6*objects['b'][i],
                angle=objects['theta'][i] * 180. / np.pi)
    e.set_facecolor('none')
    e.set_edgecolor('red')
    ax.add_artist(e)

Create arrays for the fluxes and errors of each object and print the first 10

In [ ]:
flux, fluxerr, flag = sep.sum_circle(data_sub, objects['x'], objects['y'],
                                     3.0, err=bkg.globalrms, gain=1.0)

In [ ]:
# show the first 10 objects results:
for i in range(10):
    print("object {:d}: flux = {:f} +/- {:f}".format(i, flux[i], fluxerr[i]))

Create a histogram of the fluxes

In [ ]:
image_hist = plt.hist(flux, bins='auto', range=(-0.05,0.1),fc='k', ec='k')
plt.xlabel('Fluxes')
plt.ylabel('Number of objects')